## Model the NPR articles with Latent Dirichlet Allocation

  1. Run the LDA model with sklearn (http://scikit-learn.org/stable/modules/decomposition.html#latentdirichletallocation)
  2. Visualize it with pyldavis (https://pyldavis.readthedocs.io/en/latest)

In [2]:
# USE Python3
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')

import pyLDAvis
import pyLDAvis.sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

pyLDAvis.enable_notebook()

df = pd.read_csv('npr_articles.csv', parse_dates=['date_published'])
text = df['processed_text'].values.tolist()

ImportError: No module named 'sklearn'

## Vectorize the words

Essentially create a numeric representation of the words based on frequencies

In [ ]:
max_features = 1000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=max_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(text)
print("ready")

## Run LDA

In [ ]:
n_topics = 6
lda_model = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                      learning_method='online',
                                      learning_offset=50.,
                                      random_state=0)

lda_model.fit(tf)
pyLDAvis.sklearn.prepare(lda_model,tf, tf_vectorizer, R=20)

In [ ]:
## create a matrix of the top words used to define each topic
top_words = 15
tf_feature_names = tf_vectorizer.get_feature_names() 
print_top_words(lda_model,tf_feature_names,top_words)
top_words = get_top_words(lda_model,tf_feature_names,top_words)
all_top_words = set().union(*[v for v in top_words.values()])

for key,vals in top_words.items():
    print(key," ".join(vals))
print(all_top_words)
print("total words: %s"%len(all_top_words))

In [ ]:
def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        _top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        top_words[str(topic_idx)] = _top_words
    return(top_words)

In [ ]:
topic_probas = np.zeros((max_features,n_topics),)
for topic_idx, topic in enumerate(lda_model.components_):
    topic_probas[:,topic_idx] = topic
print(topic_probas.shape)

token_probas = lda_model.transform(tf)
print(token_probas.shape)
print(tf.shape)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import preprocessing
import matplotlib.pyplot as plt

def make_scatter(fit,ax,pcX=0,pcY=1,font_size=10,font_name='sans serif',ms=20,leg=True,title=None):
    colors = ['k','cyan','r','orange','g','b','magenta']
    #cvNames = np.sort(np.unique(covs[covariate]))
    lines = []
    #for _i,i in enumerate(cvNames):
    #    indices = np.where(covs[covariate]==i)[0]
    #    s = ax.scatter(fit[indices,pcX],fit[indices,pcY],c=colors[_i],s=ms,label=covariate,alpha=0.9)
    #    lines.append(s)
    indices = np.arange(fit.shape[0])
    s = ax.scatter(fit[indices,pcX],fit[indices,pcY],s=ms,alpha=0.9)
    lines.append(s)

    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(font_size-2)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(font_size-2)

    buff = 0.02
    bufferX = buff * (fit[:,pcX].max() - fit[:,pcX].min())
    bufferY = buff * (fit[:,pcY].max() - fit[:,pcY].min())
    ax.set_xlim([fit[:,pcX].min()-bufferX,fit[:,pcX].max()+bufferX])
    ax.set_ylim([fit[:,pcY].min()-bufferY,fit[:,pcY].max()+bufferY])
    ax.set_xlabel("D-%s"%str(pcX+1),fontsize=font_size,fontname=font_name)
    ax.set_ylabel("D-%s"%str(pcY+1),fontsize=font_size,fontname=font_name)
    plt.locator_params(axis='x',nbins=5)
    ax.set_aspect(1./ax.get_data_ratio())
         
    if title:
        ax.set_title(title,fontsize=font_size+2,fontname=font_name)
    #if leg:
    #    legend = ax.legend(lines,cvNames,loc='upper right',scatterpoints=1,
    #                       handletextpad=0.01,labelspacing=0.01,borderpad=0.1,handlelength=1.0)
    #
    #    for label in legend.get_texts():
    #        label.set_fontsize(font_size-2)
    #        label.set_fontname(font_name)


fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)

mat = token_probas
matScaled = preprocessing.scale(mat)
pca_fit = PCA(n_components=2).fit_transform(mat)

make_scatter(pca_fit,ax)
plt.show()